In [24]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from src.types import Self, Point, Points, Vector, Push, Pushes
from src.board import Board
from src.solver import BoardHistory, Solver, print_solve_sequence

**Solver to do**
- Clean up the solver
    - Make the solver have an internal solving state that it updates during solving
    - Have a solve_step() method as the "core" thing, with a solve_run() that loops things
    - You can specify a timeout or max steps on the solve_run()
    - And maybe also the option to print debug stuff with rate limiting?
- Create constructors for Board objects, ideally something interactive! Like, a lightweight web app that serves as an editor.
- Come up with a format for putting problems in jsons
- Get the solver working on the "11" problem
    - "Working" = "In well under 1 second"
    - Maybe start with a few other demo puzzles
    - Might want the ability to create heuristics? Like by subclassing to specify how the queue is implemented.
    - So replace the queue list with a generic queue object (priority queue?...) that different solver implement differently.
    - Ex. heuristic: past a depth of len(board.boxes)?
    - See import queue; queue.PriorityQueue
- Create unit tests (runs solver on some test problems)

**Generator to do**
- Learn the papers
    - https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=af6c8e280946f019c7f632c8281a35ddf1fdc9b0
    - https://ianparberry.com/techreports/LARC-2011-01.pdf
    - Also do a bit more literature search
- Start implementing techniques

## Basic demos

In [27]:
# demo
airs = {(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)}
boxes = {(0, 2)}
goals = {(0, 2)}
player = (0, 0)

b = Board(airs, boxes, goals, player)
print('Player component: ', b.player_component)
print('Avaiable pushes: ', b.possible_pushes)
print('Is solved?: ', b.solved)

Player component:  {(0, 1), (0, 0)}
Avaiable pushes:  [((0, 1), (0, 1), (0, 2), (0, 3))]
Is solved?:  True


In [28]:
push = ((0, 1), (0, 1), (0, 2), (0, 3))
b2 = b.push(push)
print(repr(b2))
print('Is solved?: ', b2.solved)

Board({(0, 1), (0, 2), (0, 0), (0, 3), (1, 3)}, {(0, 3)}, {(0, 2)}, (0, 2))
Is solved?:  False


In [29]:
b == b

True

In [30]:
b == b2

False

In [31]:
hash(b), hash(b2)

(1662409248336332781, -1887771198035335534)

In [32]:
# epic
airs = {(0, 0), (0, 1), (0, 2), (0, 3), (1, 3)}
boxes = {(0, 2)}
goals = {(0, 2)}

player1 = (0, 0)
player2 = (0, 1)

b1 = Board(airs, boxes, goals, player1)
b2 = Board(airs, boxes, goals, player2)

b1 == b2

True

In [33]:
hash(b1), hash(b2)

(1662409248336332781, 1662409248336332781)

In [34]:
# trivial puzzle
airs = {(0, 0), (0, 1), (0, 2)}
boxes = {(0, 2)}
goals = {(0, 2)}
player = (0, 0)

board = Board(airs, boxes, goals, player)
solver = Solver(board)
solver.solve()

BoardHistory(Board({(0, 1), (0, 2), (0, 0)}, {(0, 2)}, {(0, 2)}, (0, 0)), [])

In [35]:
print(board)

█ █ █ █ █
█ █ █ █ █
█ █ ■ █ █
█ █   █ █
█ █ x █ █
█ █ █ █ █
█ █ █ █ █


In [36]:
# one-push puzzle
airs = {(0, 0), (0, 1), (0, 2)}
boxes = {(0, 1)}
goals = {(0, 2)}
player = (0, 0)

board = Board(airs, boxes, goals, player)
solver = Solver(board)
solver.solve()

BoardHistory(Board({(0, 1), (0, 2), (0, 0)}, {(0, 2)}, {(0, 2)}, (0, 1)), [((0, 1), (0, 0), (0, 1), (0, 2))])

In [37]:
print(board)

█ █ █ █ █
█ █ █ █ █
█ █ · █ █
█ █ □ █ █
█ █ x █ █
█ █ █ █ █
█ █ █ █ █


In [38]:
# simple puzzle
airs = {
    (1, 3), (2, 3), (3, 3),
    (1, 2), (2, 2), (3, 2),
    (1, 1), (2, 1), (3, 1),
}
boxes = {(1, 2), (2, 1), (2, 2)}
goals = {(1, 3), (3, 1), (2, 3)}
player = (1, 1)

board = Board(airs, boxes, goals, player)
solver = Solver(board)
boardhistory = solver.solve()
boardhistory.pushes

[((1, 0), (1, 1), (2, 1), (3, 1)),
 ((0, 1), (2, 1), (2, 2), (2, 3)),
 ((0, 1), (1, 1), (1, 2), (1, 3))]

In [39]:
print(board)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █ x □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █


In [40]:
print_solve_sequence(board, boardhistory.pushes)

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █ x □ · █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ·   █ █
█ █ □ □   █ █
█ █   x ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ · ■   █ █
█ █ □ x   █ █
█ █     ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █

█ █ █ █ █ █ █
█ █ █ █ █ █ █
█ █ ■ ■   █ █
█ █ x     █ █
█ █     ■ █ █
█ █ █ █ █ █ █
█ █ █ █ █ █ █


## Real puzzles

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=1

In [41]:
airs_arr = [
    [1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1],
    [0, 1, 1, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 0, 0],
]
airs_arr = list(reversed(airs_arr))

airs = set()
for i in range(len(airs_arr[0])):
    for j in range(len(airs_arr)):
        if airs_arr[j][i] == 1:
            airs.add((i+1, j+1))

boxes = {(2, 3), (3, 3)}
goals = {(3, 3), (3, 8)}
player = (3, 2)

board = Board(airs, boxes, goals, player)
print(board)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █   □ ■       █ █   █ █
█ █ █   x █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [42]:
solver = Solver(board)
boardhistory = solver.solve()
boardhistory

BoardHistory(Board({(3, 4), (4, 3), (3, 7), (5, 4), (5, 1), (9, 2), (9, 5), (2, 2), (1, 6), (2, 5), (1, 3), (1, 9), (6, 2), (7, 1), (6, 5), (3, 3), (3, 9), (3, 6), (5, 3), (8, 2), (8, 5), (9, 4), (2, 4), (1, 5), (6, 1), (1, 8), (3, 2), (3, 5), (5, 2), (3, 8), (5, 5), (9, 3), (2, 3), (2, 9), (1, 7), (7, 2), (7, 5), (6, 3)}, {(3, 8), (3, 3)}, {(3, 8), (3, 3)}, (3, 7)), [((0, 1), (2, 2), (2, 3), (2, 4)), ((0, 1), (2, 3), (2, 4), (2, 5)), ((1, 0), (1, 5), (2, 5), (3, 5)), ((0, 1), (3, 4), (3, 5), (3, 6)), ((0, 1), (3, 5), (3, 6), (3, 7)), ((0, 1), (3, 6), (3, 7), (3, 8))])

In [43]:
print_solve_sequence(board, boardhistory.pushes)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █     █   █ █ █   █ █
█ █   □ ■       █ █   █ █
█ █ █   x █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █       █           █ █
█ █ █ □   █   █ █ █   █ █
█ █   x ■       █ █   █ █
█ █ █     █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █       █ █ █ █ █ █ █ █
█ █   █ · █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   █   █ █ █ █ █ █ █ █
█ █   □   █           █ █
█ █ █ x   █   █ █ █   █ █
█ █     ■       █ █   █ █
█ █ █     █           █ █
█ █ █ █ █ █       █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ 

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=11

In [44]:
airs = {
    (4, 7),
    (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6),
    (2, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5),
    (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4),
    (1, 3), (2, 3), (3, 3), (4, 3),
    (1, 2), (4, 2),
    (1, 1), (2, 1), (3, 1), (4, 1),
}
boxes = {(3, 3), (3, 4), (2, 4), (5, 4), (5, 5), (7, 5)}
goals = {(1, 1), (1, 4), (3, 6), (5, 4), (6, 4), (6, 5)}
player = (2, 1)

board = Board(airs, boxes, goals, player)
print(board)

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · x     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █


In [ ]:
# unable to find the soln in reasonable time
solver = Solver(board)
boardhistory = solver.solve()
boardhistory.pushes

In [ ]:
print_solve_sequence(board, boardhistory.pushes)

## Bottom